<a href="https://colab.research.google.com/github/abkerper/DS4002_Project2/blob/main/draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! git clone https://github.com/abkerper/DS4002_Project2

fatal: destination path 'DS4002_Project2' already exists and is not an empty directory.


In [8]:
import numpy as np
import pandas as pd
import datetime
from pandas import read_csv
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

In [9]:
df_type = pd.read_csv('/content/SA_0000001400.csv')
df_sex = pd.read_csv('/content/SA_0000001688.csv')

In [10]:
df_total = pd.concat([df_type,df_sex])

In [11]:
display(df_total.columns)

Index(['Id', 'IndicatorCode', 'SpatialDimension', 'SpatialDimensionValueCode',
       'ParentLocationCode', 'ParentLocation', 'TimeDimension', 'TimeDim',
       'DisaggregatingDimension1', 'DisaggregatingDimension1ValueCode',
       'DisaggregatingDimension2', 'DisaggregatingDimension2ValueCode',
       'DisaggregatingDimension3', 'DisaggregatingDimension3ValueCode',
       'DataSourceDimension', 'DataSourceDimensionValueCode', 'Value',
       'NumericValue', 'Low', 'High', 'Comments', 'Date', 'TimeDimensionValue',
       'TimeDimensionBegin', 'TimeDimensionEnd'],
      dtype='object')

In [12]:
df_type = df_type[df_type['SpatialDimension']=='COUNTRY']

In [15]:
df_type = df_type[df_type['DisaggregatingDimension1ValueCode']=='ALCOHOLTYPE_SA_TOTAL']

In [16]:
display(df_type.head())

,Id,IndicatorCode,SpatialDimension,SpatialDimensionValueCode,ParentLocationCode,ParentLocation,TimeDimension,TimeDim,DisaggregatingDimension1,DisaggregatingDimension1ValueCode,...,DataSourceDimensionValueCode,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
8,1133,SA_0000001400,COUNTRY,GIN,AFR,Africa,YEAR,1966,ALCOHOLTYPE,ALCOHOLTYPE_SA_TOTAL,...,DATASOURCE_GISAHGIN,0.16,0.16,NaN,NaN,NaN,2018-05-11T09:14:17Z,1966,1966-01-01,1966-12-31
15,2745,SA_0000001400,COUNTRY,POL,EUR,Europe,YEAR,1991,ALCOHOLTYPE,ALCOHOLTYPE_SA_TOTAL,...,DATASOURCE_GISAHPOL,8.63,8.63,NaN,NaN,NaN,2018-05-11T09:17:34Z,1991,1991-01-01,1991-12-31
18,3452,SA_0000001400,COUNTRY,SVN,EUR,Europe,YEAR,1984,ALCOHOLTYPE,ALCOHOLTYPE_SA_TOTAL,...,DATASOURCE_GISAHSVN,17.23,17.23,NaN,NaN,NaN,2018-05-11T09:21:59Z,1984,1984-01-01,1984-12-31
34,5920,SA_0000001400,COUNTRY,VEN,AMR,Americas,YEAR,1973,ALCOHOLTYPE,ALCOHOLTYPE_SA_TOTAL,...,DATASOURCE_GISAHVEN,5.61,5.61,NaN,NaN,NaN,2018-05-11T09:08:11Z,1973,1973-01-01,1973-12-31
43,7475,SA_0000001400,COUNTRY,DJI,EMR,Eastern Mediterranean,YEAR,1992,ALCOHOLTYPE,ALCOHOLTYPE_SA_TOTAL,...,DATASOURCE_GISAHDJI,0.60,0.60,NaN,NaN,NaN,2018-05-11T09:11:48Z,1992,1992-01-01,1992-12-31


In [18]:
df_country = df_type[['SpatialDimensionValueCode','NumericValue','TimeDimensionValue']]

In [26]:
countries = ['AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP','SWE','GBR','USA']

In [27]:
df_country = df_country[df_country['SpatialDimensionValueCode'].isin(countries)]

In [28]:
display(df_country['SpatialDimensionValueCode'].unique())

array(['POL', 'SVN', 'MLT', 'FIN', 'PRT', 'ITA', 'IRL', 'ROU', 'CYP',
       'AUT', 'BEL', 'SVK', 'ESP', 'CZE', 'NLD', 'LVA', 'DEU', 'FRA',
       'SWE', 'USA', 'HUN', 'EST', 'GBR', 'HRV', 'GRC', 'LUX', 'BGR',
       'DNK', 'LTU'], dtype=object)

In [30]:
df_pivot = df_country.pivot(index='TimeDimensionValue', columns='SpatialDimensionValueCode', values='NumericValue')

df_pivot.reset_index(inplace=True)

In [31]:
display(df_pivot.head())

SpatialDimensionValueCode,TimeDimensionValue,AUT,BEL,BGR,CYP,CZE,DEU,DNK,ESP,EST,...,LVA,MLT,NLD,POL,PRT,ROU,SVK,SVN,SWE,USA
0,1960,8.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.83
1,1961,9.31,NaN,NaN,5.15,9.00,11.03,6.54,NaN,NaN,...,NaN,1.99,3.97,6.27,15.02,5.70,9.28,NaN,6.04,7.80
2,1962,9.70,NaN,NaN,4.44,9.12,12.05,6.56,14.64,NaN,...,NaN,1.51,4.29,6.06,15.17,6.92,9.44,NaN,6.10,7.99
3,1963,9.80,9.88,7.25,5.14,9.06,12.85,6.65,13.56,NaN,...,NaN,1.71,4.63,6.22,18.49,7.01,9.35,NaN,6.03,8.14
4,1964,10.70,10.48,6.52,5.22,9.36,13.71,7.23,13.88,NaN,...,NaN,2.07,4.97,6.16,19.37,7.03,10.21,NaN,6.36,8.44


In [ ]:
countries = df_pivot.columns[1:]  # Exclude 'Year' column

# Loop over each country and fit an ARIMA model
for country in countries:
    # Fit ARIMA model for each country's time series
    model = ARIMA(df_pivot[country], order=(p, d, q))  # Specify appropriate (p, d, q) values
    model_fit = model.fit()

    # Forecast future consumption (next 5 years as an example)
    forecast = model_fit.forecast(steps=5)

    # Plot the original data and forecast
    plt.figure(figsize=(10, 6))
    plt.plot(df['Year'], df[country], label=f"Actual {country}")
    plt.plot(range(df['Year'].iloc[-1]+1, df['Year'].iloc[-1]+6), forecast, label=f"Forecast {country}", linestyle='--')
    plt.title(f"ARIMA Forecast for {country}")
    plt.xlabel("Year")
    plt.ylabel("Alcohol Consumption")
    plt.legend()
    plt.show()


In [39]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

# Step 3: Load your dataset
# Assuming your dataset is in a CSV file
# Each column represents a country's alcohol consumption over time

# Step 4: Check if each time series is stationary and apply differencing if needed
def check_stationarity(timeseries):
    diff_df = df_pivot.copy()
    for column in df_pivot.columns[1:]:  # Assuming the first column is 'Year' or index
        # Replace inf and NaN values with a suitable value (e.g., mean)
        df_pivot[column] = df_pivot[column].replace([np.inf, -np.inf], np.nan).fillna(df_pivot[column].mean())

        p_value = check_stationarity(df_pivot[column])
        if p_value > 0.05:  # If the p-value is > 0.05, the series is non-stationary
            diff_df[column] = df_pivot[column].diff().dropna()  # Apply differencing
    return diff_df.dropna()

# Apply differencing if needed
def make_stationary(df):
    diff_df = df.copy()
    for column in df.columns[1:]:  # Assuming the first column is 'Year' or index
        p_value = check_stationarity(df[column])
        if p_value > 0.05:  # If the p-value is > 0.05, the series is non-stationary
            diff_df[column] = df[column].diff().dropna()  # Apply differencing
    return diff_df.dropna()

# Step 5: Make the data stationary
stationary_df = make_stationary(df_pivot.set_index('TimeDimensionValue'))

# Step 6: Fit the VAR model
model = VAR(stationary_df)
# Select the optimal lag order based on AIC or BIC
lag_order = model.select_order(maxlags=15)
print(f"Optimal lag order: {lag_order.selected_orders['aic']}")  # Can also use 'bic'

# Fit the model with the optimal lag
var_model_fitted = model.fit(lag_order.aic)

# Step 7: Make forecasts (next 5 periods as an example)
n_forecast = 5
forecast = var_model_fitted.forecast(stationary_df.values[-lag_order.aic:], steps=n_forecast)

# Convert forecasted values back to original scale (if differencing was applied)
forecast_df = pd.DataFrame(forecast, index=range(df_pivot['TimeDimensionValue'].iloc[-1]+1, df_pivot['TimeDimensionValue'].iloc[-1]+1+n_forecast), columns=stationary_df.columns)

# Step 8: Plot the forecasted values alongside actual values
plt.figure(figsize=(12, 6))
for column in stationary_df.columns:
    plt.plot(df_pivot['TimeDimensionValue'], df_pivot[column], label=f"Actual {column}", marker='o')
    plt.plot(forecast_df.index, forecast_df[column], label=f"Forecast {column}", linestyle='--')

plt.title("VAR Model: Forecasting Alcohol Consumption by Country")
plt.xlabel("Year")
plt.ylabel("Alcohol Consumption")
plt.legend()
plt.show()


RecursionError: maximum recursion depth exceeded while calling a Python object